# Pain in the Net
Application of Deep Image Quality Transfer (DIQT) with domain adaptation.


Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

---

References:

* `R. Tanno et al., “Uncertainty modelling in deep learning for safer neuroimage enhancement: Demonstration in diffusion MRI,” NeuroImage, vol. 225, p. 117366, Jan. 2021, doi: 10.1016/j.neuroimage.2020.117366.`
* `D. C. Alexander et al., “Image quality transfer and applications in diffusion MRI,” NeuroImage, vol. 152, pp. 283–298, May 2017, doi: 10.1016/j.neuroimage.2017.02.089.`
* `S. B. Blumberg, R. Tanno, I. Kokkinos, and D. C. Alexander, “Deeper Image Quality Transfer: Training Low-Memory Neural Networks for 3D Images,” in Medical Image Computing and Computer Assisted Intervention – MICCAI 2018, Cham, 2018, pp. 118–125, doi: 10.1007/978-3-030-00928-1_14.`


## Imports & Environment Setup

### Imports

In [ ]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 2

# imports
import collections
import functools
import io
import datetime
import time
import math
import itertools
import os
import shutil
import pathlib
import copy
import pdb
import inspect
import random
import subprocess
import sys
import warnings
from pathlib import Path
import typing
import zipfile

import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import ipyplot

# Data management libraries.
import nibabel as nib
import natsort
from natsort import natsorted
from addict import Addict
import pprint
from pprint import pprint as ppr
import box
from box import Box

# Computation & ML libraries.
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchio
import pytorch_lightning as pl
import monai

import skimage
import skimage.feature
import skimage.filters
import skimage.measure
import scipy

import pitn

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, threshold=100, linewidth=88)
torch.set_printoptions(sci_mode=False, threshold=100, linewidth=88)

In [ ]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = "direnv exec {} /usr/bin/env".format(os.getcwd())
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [ ]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

### Specs Recording

In [ ]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():
    # GPU information
    try:
        gpu_info = pitn.utils.system.get_gpu_specs()
        print(gpu_info)
    except NameError:
        print("CUDA Version: ", torch.version.cuda)
else:
    print("CUDA not in use, falling back to CPU")

In [ ]:
# cap is defined in an ipython magic command
print(cap)

Author: Tyler Spears

Last updated: 2022-01-21T23:22:56.529475+00:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.23.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-91-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 3dfe7906e10b0b4c67b0346bf50390675f5c60db

pytorch_lightning: 1.5.9
json             : 2.0.9
nibabel          : 3.2.1
pandas           : 1.2.3
torch            : 1.10.0
matplotlib       : 3.4.1
dipy             : 1.4.1
scipy            : 1.5.3
box              : 5.4.1
sys              : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
skimage          : 0.18.1
seaborn          : 0.11.1
monai            : 0.7.dev2138
numpy            : 1.20.2
pitn             : 0.0.post1.dev132+g02c0d1a
natsort          : 7.1.1
torchio          : 0.18.30

==================================================GPU Specs==================================================
  id  Name         

### Data Variables & Definitions Setup

In [ ]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"])
assert data_dir.exists()
write_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"])
assert write_data_dir.exists()
results_dir = pathlib.Path(os.environ["RESULTS_DIR"])
assert results_dir.exists()
tmp_results_dir = pathlib.Path(os.environ["TMP_RESULTS_DIR"])
assert tmp_results_dir.exists()

### Experiment Logging Setup

In [ ]:
# tensorboard experiment logging setup.
EXPERIMENT_NAME = "debug_dev_diqt_refactor"

ts = datetime.datetime.now().replace(microsecond=0).isoformat()
# Break ISO format because many programs don't like having colons ':' in a filename.
ts = ts.replace(":", "_")
experiment_name = ts + "__" + EXPERIMENT_NAME
run_name = experiment_name
# experiment_results_dir = results_dir / experiment_name

# Create temporary directory for results directory, in case experiment does not finish.
tmp_dirs = list(tmp_results_dir.glob("*"))

# Only keep up to N tmp results.
n_tmp_to_keep = 3
if len(tmp_dirs) > (n_tmp_to_keep - 1):
    print(f"More than {n_tmp_to_keep} temporary results, culling to the most recent")
    tmps_to_delete = natsorted([str(tmp_dir) for tmp_dir in tmp_dirs])[
        : -(n_tmp_to_keep - 1)
    ]
    for tmp_dir in tmps_to_delete:
        shutil.rmtree(tmp_dir)
        print("Deleted temporary results directory ", tmp_dir)

experiment_results_dir = tmp_results_dir / experiment_name
# Final target directory, to be made when experiment is complete.
final_experiment_results_dir = results_dir / experiment_name

In [ ]:
print(experiment_name)

2022-01-21T23_22_56__debug_dev_diqt_refactor


In [ ]:
# Pass this object into the pytorchlightning Trainer object, for easier logging within
# the training/testing loops.
pl_logger = pl.loggers.TensorBoardLogger(
    tmp_results_dir,
    name=experiment_name,
    version="",
    log_graph=False,
    default_hp_metric=False,
)
# Use the lower-level logger for logging histograms, images, etc.
logger = pl_logger.experiment

# Create a separate txt file to log streams of events & info besides parameters & results.
log_txt_file = Path(logger.log_dir) / "log.txt"
with open(log_txt_file, "a+") as f:
    f.write(f"Experiment Name: {experiment_name}\n")
    f.write(f"Timestamp: {ts}\n")
    # cap is defined in an ipython magic command
    f.write(f"Environment and Hardware Info:\n {cap}\n\n")

## Parameters and Function Definitions

### Parameters

In [ ]:
params = Box(default_box=True)

# General experiment-wide params
# 6 channels for the 6 DTI components
params.n_channels = 6
params.n_subjs = 15
params.lr_vox_size = 2.5
params.fr_vox_size = 1.25

# Data params
params.data.fr_dir = data_dir / f"scale-{params.fr_vox_size:.2f}mm"
params.data.lr_dir = data_dir / f"scale-{params.lr_vox_size:.2f}mm"
params.data.dti_fname_pattern = r"sub-*dti.nii.gz"
params.data.mask_fname_pattern = r"dti/sub-*mask.nii.gz"
params.data.anat_type = "t2w"
params.data.anat_fname_pattern = f"sub-*{params.data.anat_type}.nii.gz"
# The data were downsampled artificially by this factor.
params.data.downsampled_by_factor = params.lr_vox_size / params.fr_vox_size
params.data.downsampled_by_factor = (
    int(params.data.downsampled_by_factor)
    if int(params.data.downsampled_by_factor) == params.data.downsampled_by_factor
    else params.data.downsampled_by_factor
)

# This is the number of voxels to remove (read: center crop out) from the network's
# prediction. This allows for an "oversampling" of the low-res voxels to help inform a
# more constrained HR prediction. This value of voxels will be removed from each spatial
# dimension (D, H, and W) starting at the center of the output patches.
# Ex. A size of 1 will remove the 2 outer-most voxels from each dimension in the output,
# while still keeping the corresponding voxels in the LR input.
params.hr_center_crop_per_side = 5

# Scale input data by the valid values of each channel of the DTI.
# I.e., Dx,x in [0, 1], Dx,y in [-1, 1], Dy,y in [0, 1], Dy,z in [-1, 1], etc.
# params.data_scale_range = ((0, -1, 0, -1, -1, 0), (1, 1, 1, 1, 1, 1))

# Network params.
# The network's goal is to upsample the input by this factor.
params.net.upscale_factor = params.data.downsampled_by_factor
params.net.kwargs.n_res_units = 3
params.net.kwargs.n_dense_units = 3
params.net.kwargs.activate_fn = F.elu
params.net.kwargs.upsample_activate_fn = F.elu
params.net.kwargs.center_crop_output_side_amt = params.hr_center_crop_per_side

# Adam optimizer kwargs
params.optim.name = "Adam"
params.optim.kwargs.lr = 7e-4
params.optim.kwargs.betas = (0.9, 0.999)

# Testing params
params.test.dataset_subj_percent = 0.5

# Validation params
params.val.dataset_subj_percent = 0.01

# Training params
params.train.in_patch_size = (24, 24, 24)
params.train.batch_size = 16
params.train.samples_per_subj_per_epoch = 8000
params.train.max_epochs = 50
params.train.loss_name = "mse"
# Percentage of subjs in dataset that go into the training set.
params.train.dataset_subj_percent = 1 - (
    params.test.dataset_subj_percent + params.val.dataset_subj_percent
)

# If a config file exists, override the defaults with those values.
try:
    config_fname = pitn.utils.system.get_file_glob_unique(Path("."), r"config.*")
    f_type = config_fname.suffix.casefold()
    if f_type in {"yaml", "yml"}:
        f_params = Box.from_yaml(config_fname)
    elif f_type == "json":
        f_params = Box.from_json(config_fname)
    elif f_type == "toml":
        f_params = Box.from_toml(config_fname)
    else:
        raise RuntimeError()

    params.merge_update(f_params)

except:
    pass

# Remove the default_box behavior now that params have been fully read in.
p = Box(default_box=False)
p.merge_update(params)
params = p

with open(log_txt_file, "a+") as f:
    f.write(pprint.pformat(params.to_dict()) + "\n")

## Data Loading

### Subject ID Selection

In [ ]:
# Find data directories for each subject.
subj_dirs: Box = Box()

selected_ids = [
    "397154",
    "224022",
    "140117",
    "751348",
    "894774",
    "156637",
    "227432",
    "303624",
    "185947",
    "810439",
    "753251",
    "644246",
    "141422",
    "135528",
    "103010",
    "700634",
]

## Sub-set the chosen participants for dev and debugging!
sampled_subjs = random.sample(selected_ids, params.n_subjs)
if len(sampled_subjs) < len(selected_ids):
    warnings.warn(
        f"WARNING: Sub-selecting {len(sampled_subjs)}/{len(selected_ids)} "
        + "participants for dev and debugging. "
        + f"Subj IDs selected: {sampled_subjs}"
    )
selected_subjs = sampled_subjs
# ### A nested warning! For debugging only.
# warnings.warn("WARNING: Mixing training and testing subjects")
# selected_ids.append(selected_ids[0])
# ###
##

selected_subjs = natsorted(selected_subjs)

for subj_id in selected_subjs:
    subj_dirs[subj_id] = Box()
    subj_dirs[subj_id].fr = pitn.utils.system.get_file_glob_unique(
        params.data.fr_dir, f"*{subj_id}*"
    )
    subj_dirs[subj_id].lr = pitn.utils.system.get_file_glob_unique(
        params.data.lr_dir, f"*{subj_id}*"
    )
    assert subj_dirs[subj_id].fr.exists()
    assert subj_dirs[subj_id].lr.exists()
ppr(subj_dirs)

{'103010': {'fr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-1.25mm/sub-103010'),
            'lr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-2.50mm/sub-103010')},
 '135528': {'fr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-1.25mm/sub-135528'),
            'lr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-2.50mm/sub-135528')},
 '140117': {'fr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-1.25mm/sub-140117'),
            'lr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-2.50mm/sub-140117')},
 '141422': {'fr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-1.25mm/sub-141422'),
            'lr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-2.50mm/sub-141422')},
 '156637': {'fr': PosixPath('/srv/tmp/data/pitn/hcp/derivatives/mean-downsample/scale-1.25mm/sub-156637'),
            'lr': PosixPath('/srv

In [ ]:
with open(log_txt_file, "a+") as f:
    f.write(f"Selected Subjects: {selected_ids}\n")

logger.add_text("subjs", pprint.pformat(selected_ids))

### Loading and Preprocessing

In [ ]:
# Prep for Dataset loading.

# Data reader object for NIFTI files.
nib_reader = monai.data.NibabelReader(as_closest_canonical=False)

# HR -> LR patch coordinate conversion function.
fr2lr_patch_coords_fn = {
    "lr_dti": functools.partial(
        pitn.coords.transform.int_downscale_patch_idx,
        downscale_factor=params.data.downsampled_by_factor,
        downscale_patch_shape=params.train.in_patch_size,
    )
}

# Kwargs for the patches dataset (the _VolPatchDataset class) of the HR volumes.
patch_kwargs = dict(
    patch_shape=tuple(
        np.floor(
            np.asarray(params.train.in_patch_size) * params.data.downsampled_by_factor
        ).astype(int)
    ),
    stride=1,
    meta_keys_to_patch_index={"dti", "mask", params.data.anat_type},
    mask_name="mask",
)


def fix_downsample_shape_errors(
    fr_vol: torch.Tensor, fr_affine: torch.Tensor, target_spatial_shape: tuple
):
    """Small utility to fix shape differences between LR and FR data."""
    target_shape = np.asarray(target_spatial_shape)
    if fr_vol.shape[1:] != tuple(target_shape):
        # Use torchio objects because they fix the affine matrix, too.
        # Flip before transform to pad on the right/top/furthest side of the dimension
        # first, before the left/bottom/closest.
        flip_vol = fr_vol.flip([1, 2, 3])
        im = torchio.ScalarImage(tensor=flip_vol, affine=fr_affine)
        transform = torchio.transforms.CropOrPad(target_spatial_shape, 0, copy=False)
        im = transform(im)
        result_vol = im["data"]
        # Unflip.
        result_vol = result_vol.flip([1, 2, 3])
        result_aff = im["affine"]
    else:
        result_vol = fr_vol
        result_aff = fr_affine

    return result_vol, result_aff

#### Data Loading

In [ ]:
# Import and organize all data.
subj_data: dict = dict()

meta_keys_to_keep = {"affine", "original_affine"}
for subj_id, subj_dir in subj_dirs.items():

    data = dict()
    data["subj_id"] = subj_id
    fr_subj_dir = subj_dirs[subj_id]["fr"]
    lr_subj_dir = subj_dirs[subj_id]["lr"]
    data["fr_subj_dir"] = fr_subj_dir
    data["lr_subj_dir"] = lr_subj_dir

    # Low-resolution DTI.
    lr_dti_f = pitn.utils.system.get_file_glob_unique(
        lr_subj_dir, params.data.dti_fname_pattern
    )
    im = nib_reader.read(lr_dti_f)
    lr_dti, meta = nib_reader.get_data(im)
    meta = {k: meta[k] for k in meta_keys_to_keep}
    lr_dti = torch.from_numpy(lr_dti)
    data["lr_dti"] = lr_dti
    data["lr_dti_meta_dict"] = meta

    # May need to handle shape errors when re-upscaling back from LR to HR.
    lr_dti_shape = np.asarray(lr_dti.shape[1:])
    target_fr_shape = np.floor(lr_dti_shape * params.net.upscale_factor).astype(int)

    # Full-resolution images/volumes.
    # DTI.
    dti_f = pitn.utils.system.get_file_glob_unique(
        fr_subj_dir, params.data.dti_fname_pattern
    )
    im = nib_reader.read(dti_f)
    dti, meta = nib_reader.get_data(im)
    dti = torch.from_numpy(dti)
    meta = {k: meta[k] for k in meta_keys_to_keep}
    dti, meta["affine"] = fix_downsample_shape_errors(
        dti, meta["affine"], target_fr_shape
    )
    data["dti"] = dti
    data["dti_meta_dict"] = meta

    # Diffusion mask.
    mask_f = pitn.utils.system.get_file_glob_unique(
        fr_subj_dir, params.data.mask_fname_pattern
    )
    im = nib_reader.read(mask_f)
    mask, meta = nib_reader.get_data(im)
    meta = {k: meta[k] for k in meta_keys_to_keep}
    mask = torch.from_numpy(mask)
    # Add channel dim if not available.
    if mask.ndim == 3:
        mask = mask[
            None,
        ]
    mask, meta["affine"] = fix_downsample_shape_errors(
        mask, meta["affine"], target_fr_shape
    )
    mask = mask.bool()
    data["mask"] = mask
    data["mask_meta_dict"] = meta

    # Anatomical/structural volume.
    anat_f = pitn.utils.system.get_file_glob_unique(
        fr_subj_dir, params.data.anat_fname_pattern
    )
    im = nib_reader.read(anat_f)
    anat, meta = nib_reader.get_data(im)
    meta = {k: meta[k] for k in meta_keys_to_keep}
    anat = torch.from_numpy(anat)
    if anat.ndim == 3:
        anat = anat[
            None,
        ]
    anat, meta["affine"] = fix_downsample_shape_errors(
        anat, meta["affine"], target_fr_shape
    )
    data[params.data.anat_type] = anat
    data[params.data.anat_type + "_meta_dict"] = meta

    vol_names = {"dti", "mask", "lr_dti", params.data.anat_type}
    metadata_names = set(data.keys()) - vol_names
    vol_d = {k: data[k] for k in vol_names}
    meta_d = {k: data[k] for k in metadata_names}

    # Create multi-volume dataset for this subj-session.
    subj_dataset = pitn.data.SubjSesDataset(
        vol_d,
        primary_vol_name="dti",
        special_secondary2primary_coords_fns=fr2lr_patch_coords_fn,
        transform=None,
        primary_patch_kwargs=patch_kwargs,
        **meta_d
    )
    print("Creating patches")
    # Init the patches dataset.
    subj_dataset.patches

    # Finalize this subject.
    subj_data[subj_id] = subj_dataset

    print("=" * 20)

print("===Data Loaded===")

In [ ]:
# fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, dpi=120)

# sample = subj_data["103010"].patches[20000]
# ax1.imshow(sample["t2w"][0, 0, :, 0], cmap="gray", interpolation=None)
# ax2.imshow(sample["dti"][0, 0, :, 0], cmap="gray")
# ax3.imshow(sample["mask"][0, 0, :, 0], cmap="gray")
# ax4.imshow(sample["lr_dti"][0, 0, :, 0], cmap="gray");
# print(sample['t2w'].shape, sample['dti'].shape, sample['mask'].shape, sample['lr_dti'].shape)

In [ ]:
# Finalize the dataset.
# subj_dataset = torchio.SubjectsDataset(list(subj_data.values()), load_getitem=False)

## Model Training

### Set Up Patch-Based Data Loaders

In [ ]:
# Data train/validation/test split

num_subjs = len(subj_data)
num_test_subjs = int(np.ceil(params.n_subjs * params.test.dataset_subj_percent))
num_val_subjs = int(np.ceil(params.n_subjs * params.val.dataset_subj_percent))
num_train_subjs = params.n_subjs - (num_test_subjs + num_val_subjs)
subj_list = list(subj_data.keys())
# Randomly shuffle the list of subjects, then choose the first `num_test_subjs` subjects
# for testing.
random.shuffle(subj_list)

# Choose the remaining for training/validation.
# If only 1 subject is available, assume this is a debugging run.
if num_subjs == 1:
    warnings.warn(
        "DEBUG: Only 1 subject selected, mixing training, validation, and testing sets"
    )
    num_train_subjs = num_val_subjs = num_test_subjs = 1

    test_subjs = subj_list
    val_subjs = subj_list
    train_subjs = subj_list
else:
    test_subjs = subj_list[:num_test_subjs]
    val_subjs = subj_list[num_test_subjs : (num_test_subjs + num_val_subjs)]
    train_subjs = subj_list[(num_test_subjs + num_val_subjs) :]

print(f"{num_train_subjs}/{num_subjs} Training subject ID(s): {train_subjs}")
print(f"{num_val_subjs}/{num_subjs} Validation subject ID(s): {val_subjs}")
print(f"{num_test_subjs}/{num_subjs} Test subject ID(s): {test_subjs}")
with open(log_txt_file, "a+") as f:
    f.write(f"{num_train_subjs}/{num_subjs} Training subject ID(s): {train_subjs}\n")
    f.write(f"{num_val_subjs}/{num_subjs} Validation subject ID(s): {val_subjs}\n")
    f.write(f"{num_test_subjs}/{num_subjs} Test subject ID(s): {test_subjs}\n")

logger.add_text("train_subjs", str(train_subjs))
logger.add_text("val_subjs", str(val_subjs))
logger.add_text("test_subjs", str(test_subjs))

In [ ]:
# Create Datasets and DataLoaders for test, validation, and training steps.
sample_kws = {
    "subj_id": "subj_id",
    "dti": "dti",
    "lr_dti": "lr_dti",
    "mask": "mask",
    params.data.anat_type: params.data.anat_type,
}

# Train
train_ds = list()
for subj_id in train_subjs:
    train_ds.append(subj_data[subj_id].patches)
train_dataset = torch.utils.data.ConcatDataset(train_ds)
train_sampler = pitn.samplers.ConcatDatasetBalancedRandomSampler(
    train_dataset.datasets,
    max_samples_per_dataset=params.train.samples_per_subj_per_epoch,
)
train_collate_fn = functools.partial(pitn.samplers.collate_dicts, **sample_kws)
train_loader = monai.data.DataLoader(
    train_dataset,
    sampler=train_sampler,
    batch_size=params.train.batch_size,
    collate_fn=train_collate_fn,
    pin_memory=True,
    num_workers=7,
    persistent_workers=True,
)

# Test & Validation
test_val_collate_fn = functools.partial(pitn.samplers.collate_dicts, **sample_kws)

test_ds = list()
for subj_id in test_subjs:
    test_ds.append(subj_data[subj_id])
test_dataset = torch.utils.data.ConcatDataset(test_ds)
test_loader = monai.data.DataLoader(
    test_dataset, collate_fn=test_val_collate_fn, batch_size=1
)

val_ds = list()
for subj_id in val_subjs:
    val_ds.append(subj_data[subj_id])
val_dataset = torch.utils.data.ConcatDataset(val_ds)
val_loader = monai.data.DataLoader(
    val_dataset, collate_fn=test_val_collate_fn, batch_size=1
)

### Bounding Box Selection for Visualization During Training

In [ ]:
# # Choose only one subject for visualization during validation.
# val_viz_subj_id = random.choice([s.subj_id for s in val_dataset])
# # Store its index in the Dataset object for easier indexing.
# val_viz_dataset_idx = [s.subj_id for s in val_dataset].index(val_viz_subj_id)
# print(val_viz_subj_id)

In [ ]:
# # Create bbox coordinates for visualizing validation aggregate patches.
# # NOTE: This presumes that there are no overlap patches in the validation step, and that
# # each validation volume is divisible by the patch shape in each spatial dimension.
# bbox_coords = list()
# region_size = torch.as_tensor(output_spatial_patch_shape) * 3
# vol_shape = torch.as_tensor(val_dataset[val_viz_dataset_idx].fr_dti.data.shape[1:])
# possible_bbox_ini = [
#     torch.arange(0, vol_shape[0], output_spatial_patch_shape[0]),
#     torch.arange(0, vol_shape[1], output_spatial_patch_shape[1]),
#     torch.arange(0, vol_shape[2], output_spatial_patch_shape[2]),
# ]

# # Create bbox that spans roughly the center of the volume.
# bbox_idx_ini = list()
# for possible_bbox_part in possible_bbox_ini:
#     num_parts = len(possible_bbox_part)
#     bbox_idx_ini.append(possible_bbox_part[round(num_parts * 0.4)])
# bbox_idx_ini = torch.as_tensor(bbox_idx_ini)

# bbox_coord = torch.cat([bbox_idx_ini, bbox_idx_ini + region_size])
# bbox_coords.append(bbox_coord)

In [ ]:
# # Create bbox that covers an edge.
# vol_mask = val_dataset[val_viz_dataset_idx].fr_brain_mask.data[0]
# bbox_mask_coverage = list()
# possible_bbox_coords_ini = list(itertools.product(*possible_bbox_ini))
# for bbox_w, bbox_h, bbox_d in possible_bbox_coords_ini:
#     bbox_start = (bbox_w, bbox_h, bbox_d)
#     bbox_end = (
#         bbox_start[0] + output_spatial_patch_shape[0],
#         bbox_start[1] + output_spatial_patch_shape[1],
#         bbox_start[2] + output_spatial_patch_shape[2],
#     )
#     patch = vol_mask[
#         bbox_start[0] : bbox_end[0],
#         bbox_start[1] : bbox_end[1],
#         bbox_start[2] : bbox_end[2],
#     ]
#     # Validation visuals are shown over the Superior-Anterior axis, so the edge should be
#     # located there.
#     bbox_mask_coverage.append(
#         patch[:, output_spatial_patch_shape[1] // 2, :].sum().item()
#     )

# # Now search all coordinates' mask coverages for the one closest to a 50% of the total
# # patch volume.
# patch_vol = output_spatial_patch_shape[0] * output_spatial_patch_shape[2]
# target_mask_vol = patch_vol // (2 ** 2)

# bbox_coord_idx = np.argmin(np.abs(np.asarray(bbox_mask_coverage) - patch_vol))
# bbox_idx_ini = torch.tensor(possible_bbox_coords_ini[bbox_coord_idx])
# bbox_coord = torch.cat([bbox_idx_ini, bbox_idx_ini + region_size])
# bbox_coords.append(bbox_coord)

In [ ]:
# bbox_coords = torch.stack(bbox_coords)

# bbox_coords

### Model Definition

In [ ]:
# Full pytorch-lightning module for contained training, validation, and testing.
debug_prob = -1 / (
    params.train.samples_per_subj_per_epoch * len(train_subjs) / params.train.batch_size
)


class DIQTCascadeSystem(pl.LightningModule):

    # Specify training loss methods with mappings to their names as strings.
    loss_methods = {
        "MSE".casefold(): torch.nn.MSELoss(reduction="mean"),
        "SSE".casefold(): torch.nn.MSELoss(reduction="sum"),
        "RMSE".casefold(): lambda y_hat, y: torch.sqrt(
            F.mse_loss(y_hat, y, reduction="mean")
        ),
        "L1".casefold(): torch.nn.L1Loss(reduction="mean"),
    }

    def __init__(
        self,
        channels,
        in_patch_shape: tuple,
        upscale_factor: int,
        anat_batch_key: str,
        train_loss_method: str,
        opt_params: dict,
        val_viz_every_n_epochs=1,
        **net_kwargs,
    ):
        super().__init__()

        self._channels = channels
        self._in_patch_shape = in_patch_shape
        self._anat_batch_key = anat_batch_key
        self._upscale_factor = upscale_factor

        # Parameters
        # Network parameters
        self.net = pitn.nn.sr.CascadeUpsampleModeRefine(
            self._channels, upscale_factor=self._upscale_factor, **net_kwargs
        )

        ## Training parameters
        self.opt_params = opt_params

        # Select loss method as either one of the pre-selected methods, or a custom
        # callable.
        try:
            self._loss_fn = self.loss_methods[train_loss_method.casefold()]
        except (AttributeError, KeyError):
            if callable(train_loss_method):
                self._loss_fn = train_loss_method
            else:
                raise ValueError(
                    f"ERROR: Invalid loss function specification {train_loss_method}, "
                    + f"expected one of {self.loss_methods.keys()} or a callable."
                )

        self._val_viz_every_n_epochs = val_viz_every_n_epochs

        # My own dinky logging object.
        self.plain_log = Box(
            {
                "train_loss": list(),
                "val_loss": {"rmse": list()},
                "test_loss": {"rmse": dict()},
                "viz": {
                    "test_preds": dict(),
                    "test_squared_error": dict(),
                },
            },
        )

    def forward(self, x, x_mode_refine):
        y = self.net(x, x_mode_refine)
        return y

    def training_step(self, batch, batch_idx):
        batch = Box(batch)
        x = batch.lr_dti
        y = batch.dti
        y = self.net.crop_full_output(y)
        x_mode_refine = batch[self._anat_batch_key]
        mask = batch.mask.bool()
        mask = self.net.crop_full_output(mask)

        y_pred = self.net(x, x_mode_refine)

        # Only calculate loss on voxels in the brain mask.
        loss = self._loss_fn(
            torch.masked_select(y_pred, mask), torch.masked_select(y, mask)
        )

        self.log("train_loss", loss)
        self.plain_log["train_loss"].append(float(loss.cpu()))
        return loss

    def validation_step(self, batch, batch_idx):

        batch = Box(batch)
        x = batch.lr_dti
        y = batch.dti
        y = self.net.crop_full_output(y)
        mask = batch.mask.bool()
        mask = self.net.crop_full_output(mask)
        x_mode_refine = batch[self._anat_batch_key]

        y_pred = self.net(x, x_mode_refine)

        rmse_loss = torch.sqrt(
            F.mse_loss(
                torch.masked_select(y_pred, mask),
                torch.masked_select(y, mask),
                reduction="mean",
            )
        )

        self.log("val_loss/rmse", rmse_loss)
        self.plain_log.val_loss.rmse.append(float(rmse_loss.cpu()))

        return rmse_loss

    def test_step(self, batch: dict, batch_idx):
        batch = Box(batch)
        subj_id = batch.subj_id[0]
        x = batch.lr_dti
        y = batch.dti
        y = self.net.crop_full_output(y)
        mask = batch.mask.bool()
        mask = self.net.crop_full_output(mask)
        x_mode_refine = batch[self._anat_batch_key]

        y_pred = self.net(x, x_mode_refine)

        rmse_loss = torch.sqrt(
            F.mse_loss(
                torch.masked_select(y_pred, mask),
                torch.masked_select(y, mask),
                reduction="mean",
            )
        )

        self.log("test_loss/rmse", rmse_loss)
        self.plain_log.test_loss.rmse[subj_id] = rmse_loss.detach().cpu().item()
        self.plain_log.viz.test_preds[subj_id] = y_pred[0].detach().cpu()

        return rmse_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), **self.opt_params)
        return optimizer

### Training Loop

In [ ]:
x = (np.arange(12) ** 2).reshape(4, 3).tolist()
nrow = 4
ncol = 3
print(x)
rows = [l[i] for i in range(ncol) for l in itertools.chain(x)]
rows = [rows[s:s+ncol] for s in range(0, nrow*ncol, ncol)]
rows
# row_heights = [sum(map(lambda im: im.shape[0] if im is not None else 0), r)) for r in rows]

In [ ]:
train_start_timestamp = datetime.datetime.now().replace(microsecond=0)

model = DIQTCascadeSystem(
    channels=params.n_channels,
    in_patch_shape=params.train.in_patch_size,
    upscale_factor=params.net.upscale_factor,
    anat_batch_key=params.data.anat_type,
    train_loss_method=params.train.loss_name,
    opt_params=params.optim.kwargs,
    val_viz_every_n_epochs=1,
    **params.net.kwargs,
)

with open(log_txt_file, "a+") as f:
    f.write(f"Model overview: {model}\n")

# Create trainer object.
trainer = pl.Trainer(
    #     fast_dev_run=10,
    gpus=1,
    max_epochs=params.train.max_epochs,
    # max_epochs=1,
    logger=pl_logger,
    log_every_n_steps=50,
    check_val_every_n_epoch=2,
    detect_anomaly=True,
)

# Many warnings are produced here, so it's better for my sanity (i.e., worse in every other
# way) to just filter and ignore them...
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

train_duration = datetime.datetime.now().replace(microsecond=0) - train_start_timestamp
print(f"Train duration: {train_duration}")

In [ ]:
# Save out trained model
trainer.save_checkpoint(str(experiment_results_dir / "model.ckpt"))

In [ ]:
with open(log_txt_file, "a+") as f:
    f.write("\n")
    f.write(f"Training time: {train_duration}\n")
    f.write(
        f"\t{train_duration.days} Days, "
        + f"{train_duration.seconds // 3600} Hours,"
        + f"{(train_duration.seconds // 60) % 60} Minutes,"
        + f'{train_duration.seconds % 60} Seconds"\n'
    )

In [ ]:
# Plot rolling average window of training loss values.
plt.figure(dpi=110)
window = 1000
rolling_mean = (
    np.convolve(model.plain_log["train_loss"], np.ones(window), "valid") / window
)
rolling_start = 100
plt.plot(
    np.arange(
        window + rolling_start,
        window + rolling_start + len(rolling_mean[rolling_start:]),
    ),
    rolling_mean[rolling_start:],
)
plt.title("Training Loss " + params.train.loss_name + f"\nRolling Mean {window}")
plt.xlabel("Epoch")
plt.ylabel("Loss")
# plt.ylim(0, 1)
print(np.median(rolling_mean))
print(
    np.mean(model.plain_log["train_loss"][: window + rolling_start]),
    np.var(model.plain_log["train_loss"][: window + rolling_start]),
    np.max(model.plain_log["train_loss"][: window + rolling_start]),
)

plt.savefig(experiment_results_dir / "train_loss.png")

## Model Testing/Evaluation

### Testing Loop

In [ ]:
# Store test reconstructions along the way for later visualization.
test_vol_viz = Addict()

# Structure is as follows:
# {subj_id_1:
#    fr_mask: np.ndarray,
#    dti: {
#          diqt: np.ndarray,
#          spline: np.ndarray,
#          fr: np.ndarray,
#          lr: np.ndarray
#          ...
#         }
#    fa: {
#         diqt: np.ndarray,
#         ...
#        },
# pitn.data.norm.denormalize_batch(y, mean=fr_means, var=fr_vars, eps=self._norm_eps)
#  subj_id_2:
#     ....
# }

In [ ]:
# # Debug code for model editing after training has been completed.

# # If the model needs to be created, a.k.a. the training loop cell wasn't executed.
# model = DIQTSystem(
#     channels=channels,
#     downsample_factor=exp_params.downsample_factor,
#     source_vox_size=exp_params.source_vox_size,
#     target_vox_size=exp_params.target_vox_size,
#     norm_method=network_norm_method,
#     train_loss_method=train_loss_name,
#     opt_params=opt_params,
#     val_viz_subj_id=val_viz_subj_id,
#     val_viz_bboxes=bbox_coords,
#     val_viz_every_n_epochs=5,
# )

# model.load_from_checkpoint(
#     experiment_results_dir / "model.ckpt",,
#     channels=channels,
#     downsample_factor=exp_params.downsample_factor,
#     source_vox_size=exp_params.source_vox_size,
#     target_vox_size=exp_params.target_vox_size,
#     norm_method=network_norm_method,
#     train_loss_method=train_loss_name,
#     opt_params=opt_params,
#     val_viz_subj_id=val_viz_subj_id,
#     val_viz_bboxes=bbox_coords,
#     val_viz_every_n_epochs=5,
# )

# trainer.test(model, dataloaders=test_loader, ckpt_path=None, verbose=True)

In [ ]:
trainer.test(model, dataloaders=test_loader, ckpt_path=None, verbose=True)

In [ ]:
plt.figure(dpi=120)
plt.imshow(model.plain_log.viz.test_preds["103010"].cpu().numpy()[2, 80])
plt.colorbar();

In [ ]:
# Store DIQT test reconstructions for visualization.
for subj_id in test_subjs:
    test_vol_viz[subj_id].dti.diqt = (
        model.plain_log.viz.test_preds[subj_id].cpu().numpy()
    )

In [ ]:
test_vol_viz.keys()

In [ ]:
test_loss_name = "RMSE"
test_loss_log_file = experiment_results_dir / "test_loss.csv"

with open(log_txt_file, "a+") as f:
    f.write(f"Test loss function: {test_loss_name}\n")

row_iter = enumerate(list(model.plain_log["test_loss"].values()))
test_loss_tabular = "".join([f"{batch_idx}, {loss}\n" for batch_idx, loss in row_iter])

with open(test_loss_log_file, "a+") as f:
    f.write("batch_idx, loss\n")
    f.write(test_loss_tabular)

In [ ]:
test_loss_tabular

### Evaluation Visualization

In [ ]:
model.plain_log.test_loss.rmse.keys()

In [ ]:
# Plot testing loss values over all patches.
fig, ax_prob = plt.subplots(figsize=(8, 4), dpi=120)
log_scale = False

hist = sns.histplot(
    list(model.plain_log["test_loss"].rmse.values()),
    alpha=0.5,
    stat="count",
    log_scale=log_scale,
    ax=ax_prob,
    legend=False,
    hatch="\\\\",
    ec="blue",
)
hist.yaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
plt.xlabel("Loss in $mm^2/second$")

# Draw means of different comparison models.
comparison_kwargs = {"ls": "-", "lw": 2.5}
# Plot the current DNN model performance.
plt.axvline(
    np.asarray(list(model.plain_log["test_loss"].rmse.values())).mean(),
    label="Current Model Mean",
    color="blue",
    **comparison_kwargs,
)

# Tanno, et. al., 2021 model comparisons.
# Taken from Table 2, HCP exterior.
plt.axvline(
    31.738e-4,
    label="(Tanno etal, 2021) C-spline Mean",
    color="red",
    **comparison_kwargs,
)
plt.axvline(
    23.139e-4,
    label="(Tanno etal, 2021) RF",
    color="orange",
    **comparison_kwargs,
)
plt.axvline(
    13.609e-4,
    label="(Tanno etal, 2021) ESPCN Baseline",
    color="green",
    **comparison_kwargs,
)
plt.axvline(
    13.412e-4,
    label="(Tanno etal, 2021) Best",
    color="purple",
    **comparison_kwargs,
)
# Best performing Blumberg, et. al., 2018 paper model.
plt.axvline(
    12.78e-4,
    label="(Blumberg etal, 2018) Best",
    color="pink",
    **comparison_kwargs,
)

plt.legend(fontsize="small")
plt.title(f"Test Loss Histogram Over All Subjects with Test Metric {test_loss_name}")
plt.savefig(experiment_results_dir / "test_loss_hist.png")

In [ ]:
# Plot testing loss values over all subjects.
fig, ax = plt.subplots(figsize=(8, 4), dpi=120)

models = (
    "(Ours)\nCurrent Model",
    "(Tanno etal, 2021)\nC-spline Mean",
    "(Tanno etal, 2021)\nRF",
    "(Tanno etal, 2021)\nESPCN Baseline",
    "(Tanno etal, 2021)\nBest",
    "(Blumberg etal, 2018)\nBest",
)

rmse_scores = (
    np.asarray(list(model.plain_log["test_loss"].rmse.values())).mean(),
    31.738e-4,
    23.139e-4,
    13.609e-4,
    13.412e-4,
    12.13e-4,
)
rmse_std_error = np.asarray([0, 0, 0.351e-4, 0.084e-4, 0.041e-4, 1.24e-4])

ax.grid(True, axis="y", zorder=1000)
ax.set_axisbelow(True)
ax.bar(
    models,
    rmse_scores,
    yerr=rmse_std_error,
    color=sns.color_palette("deep", n_colors=len(rmse_scores)),
    edgecolor="black",
    lw=0.75,
)

for container in ax.containers:
    if isinstance(container, mpl.container.BarContainer):
        ax.bar_label(container, fmt="%.3e")

ax.set_ylim(bottom=0, top=ax.get_ylim()[1] * 1.1)
ax.set_xlabel("Model")
ax.set_ylabel("Loss in $mm^2/second$")

ax.set_title(f"Mean Over Subjects Test Loss {test_loss_name}")
ax.set_xticks(models)
ax.set_xticklabels(
    models, fontsize="x-small", rotation=25, ha="right", rotation_mode="anchor"
)
plt.savefig(experiment_results_dir / "test_loss_bar.png")

In [ ]:
print(model.plain_log["test_loss"])
print(np.mean(list(model.plain_log["test_loss"].values())))

In [ ]:
sorted_test_idx = np.argsort(np.asarray(list(model.plain_log["test_loss"].values())))
sorted_test_results = dict(
    list(model.plain_log["test_loss"].items())[i] for i in sorted_test_idx
)
ppr(sorted_test_results, sort_dicts=False)

In [ ]:
logger.add_histogram(
    "test/rmse_dist", np.asarray(list(model.plain_log["test_loss"].values()))
)

### Save out metrics and Hyperparameters

In [ ]:
compare_hparams.metric["hparam/rmse"] = np.mean(
    list(model.plain_log["test_loss"].values())
)
with open(log_txt_file, "a+") as f:
    f.write(
        f"Mean RMSE Testing Value: {np.mean(list(model.plain_log['test_loss'].values()))}\n"
    )
    f.write(f"Mean RMSE Spline Value: {spline_loss_mean}\n")

logger.add_scalar("metric/rmse", np.mean(list(model.plain_log["test_loss"].values())))
logger.add_scalar("metric/spline", spline_loss_mean)

## Whole-Volume Visualization

### Setup

In [ ]:
# Debug flag(s)
disable_fig_save = False

In [ ]:
# Create full 3D volume of full-res ground truth, low-res downsample, full-res mask, and
# full-res predictions.

with torch.no_grad():

    for subj_i in test_dataset:
        subj = copy.copy(subj_i)
        subj_id = subj["subj_id"]
        print(f"Starting subject {subj_id}")

        # Collect all variants of the volume and aggregate into one container object.
        fr_vol = subj["gt_dti"]["data"].clone()
        lr_vol = subj["lr_dti"]["data"].clone()

        full_res_predicted = torch.from_numpy(test_vol_viz[subj_id].dti.diqt.copy())
        full_res_spline = torch.from_numpy(test_vol_viz[subj_id].dti.spline.copy())

        #         warnings.warn("======== Skipping all de-normalization for visualization.")
        #         if False:
        if data_norm_method is not None and "channel" in data_norm_method.casefold():
            print("Normalizing")

            lr_means = torch.as_tensor(subj["lr_means"]).to(lr_vol).clone()
            lr_vars = torch.as_tensor(subj["lr_vars"]).to(lr_vol).clone()

            lr_vol = pitn.data.norm.denormalize_dti(
                lr_vol, mean=lr_means, var=lr_vars
            ).clone()

        # Zero-out all voxels outside the mask.

        fr_mask = subj["fr_brain_mask"]["data"].bool().clone()
        full_res_spline = full_res_spline * fr_mask.to(full_res_spline).bool()
        fr_vol = fr_vol * fr_mask.to(fr_vol).bool()
        full_res_predicted = full_res_predicted * fr_mask.to(full_res_predicted).bool()
        lr_vol = lr_vol * subj["lr_brain_mask"]["data"].to(lr_vol).bool()
        abs_error = torch.abs(full_res_predicted - fr_vol)

        test_vol_viz[subj.subj_id].dti.update(
            fr=fr_vol.cpu().numpy(),
            lr=lr_vol.cpu().numpy(),
            diqt=full_res_predicted.cpu().numpy(),
            spline=full_res_spline.cpu().numpy(),
        )

        test_vol_viz[subj.subj_id].fa.update(
            itertools.starmap(
                lambda k, v: (k, pitn.viz.fa_map(test_vol_viz[subj_id].dti[k])),
                test_vol_viz[subj_id].dti.items(),
            )
        )
        test_vol_viz[subj_id].fr_mask = fr_mask.cpu().numpy()
        test_vol_viz[subj_id].dti.abs_error = abs_error.cpu().numpy()

        print(f"Finished subject {subj_id}")

In [ ]:
# Save out all network predictions to Nifti2 files and compress them into a zip archive.
if not disable_fig_save:
    img_names = list()
    for subj_id, viz in test_vol_viz.items():
        pred_vol = viz.dti.diqt
        affine = subj_data[subj_id]["gt_dti"].affine
        nib_img = nib.Nifti2Image(pred_vol, affine)

        filename = experiment_results_dir / f"{subj_id}_predicted_dti.nii.gz"
        nib.save(nib_img, str(filename))
        img_names.append(filename)

    with zipfile.ZipFile(experiment_results_dir / "predicted_dti.zip", "w") as fzip:
        for filename in img_names:
            fzip.write(
                filename,
                arcname=filename.name,
                compress_type=zipfile.ZIP_DEFLATED,
                compresslevel=6,
            )
            os.remove(filename)
    # Make sure we exit the 'with' statement above.
    print("Done with files")

In [ ]:
viz_subj_idx = random.choice(list(range(len(test_vol_viz.keys()))))
# viz_subj_id = list(test_vol_viz.keys())[viz_subj_idx]
# Pick the worst performing subject from the test set.
viz_subj_id = list(sorted_test_results.keys())[-1]
print(list(test_vol_viz.keys()))
print(viz_subj_id)

### FA-Weighted Direction Maps

In [ ]:
# Generate FA-weighted diffusion direction map for prediction.
pred_dir_map = pitn.viz.direction_map(test_vol_viz[viz_subj_id].dti.diqt)
# Set channels last for matplotlib
pred_dir_map = pred_dir_map.transpose(1, 2, 3, 0)
test_vol_viz[viz_subj_id].color_fa.diqt = pred_dir_map

spline_dir_map = pitn.viz.direction_map(test_vol_viz[viz_subj_id].dti.spline)
# Set channels last for matplotlib
spline_dir_map = spline_dir_map.transpose(1, 2, 3, 0)
test_vol_viz[viz_subj_id].color_fa.spline = spline_dir_map

fr_dir_map = pitn.viz.direction_map(test_vol_viz[viz_subj_id].dti.fr)
# Set channels last for matplotlib
fr_dir_map = fr_dir_map.transpose(1, 2, 3, 0)
test_vol_viz[viz_subj_id].color_fa.fr = fr_dir_map

lr_dir_map = pitn.viz.direction_map(test_vol_viz[viz_subj_id].dti.lr)
# Set channels last for matplotlib
lr_dir_map = lr_dir_map.transpose(1, 2, 3, 0)
test_vol_viz[viz_subj_id].color_fa.lr = lr_dir_map

In [ ]:
slice_idx = (slice(None, None, None), slice(None, None, None), 86)
low_res_slice_idx = tuple(
    int(np.round(s / downsample_factor)) if isinstance(s, int) else s for s in slice_idx
)
print(slice_idx)
print(low_res_slice_idx)

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(pred_dir_map[slice_idx]), interpolation="none")
plt.axis("off")
plt.title("Predicted with DIQT Net")
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "pred_dir_map_sample.png");

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(fr_dir_map[slice_idx]), interpolation="none")
# plt.colorbar()
plt.axis("off")
plt.title("Ground Truth")
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "ground_truth_dir_map_sample.png")

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(spline_dir_map[slice_idx]), interpolation="none")
plt.axis("off")
plt.title(f"Spline Interpolation Order {exp_params.spline.order}")
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "cubic_spline_dir_map_sample.png");

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(lr_dir_map[low_res_slice_idx]), interpolation="none")
plt.axis("off")
plt.title("LR Input")
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "low_res_map_sample.png");

### DTI Channel-Wise Visualization

In [ ]:
channel_names = [
    "$D_{x,x}$",
    "$D_{x,y}$",
    "$D_{y,y}$",
    "$D_{x,z}$",
    "$D_{y,z}$",
    "$D_{z,z}$",
]
dti_names = [
    "Full-Res",
    "Low-Res Input",
    f"Spline Order {exp_params.spline.order}",
    "Predicted",
    "Absolute Error\nFR vs. Predicted",
]

#### Global Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, and root squared error

cmap = "jet"

dtis = [
    test_vol_viz[viz_subj_id].dti.fr[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.lr[(slice(None), *low_res_slice_idx)],
    test_vol_viz[viz_subj_id].dti.spline[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.diqt[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.abs_error[(slice(None), *slice_idx)],
]

nrows = len(dtis)
ncols = len(channel_names)

# Don't take the absolute max and min values, as there exist some extreme (e.g., > 3
# orders of magnitude) outliers. Instead, take some percente quantile.
# Reshape and concatenate the dtis in order to compute the quantiles of images with
# different shapes (e.g., the low-res input patch).
max_dti = np.quantile(np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.95)
min_dti = np.quantile(np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.05)

nrows = len(dtis)
ncols = len(channel_names)

fig = plt.figure(figsize=(12, 6), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)
axs = list()
max_subplot_height = 0
for i_row in range(nrows):
    dti = dtis[i_row]

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(
            np.rot90(dti[j_col]),
            cmap=cmap,
            interpolation=None,
            vmin=min_dti,
            vmax=max_dti,
        )
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(dti_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(channel_names[j_col])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        axs.append(ax)

color_norm = mpl.colors.Normalize(vmin=min_dti, vmax=max_dti)
fig.colorbar(
    mpl.cm.ScalarMappable(norm=color_norm, cmap=cmap),
    ax=axs,
    location="right",
    fraction=0.1,
    pad=0.03,
)
plt.suptitle(
    "DTI Channel Breakdown, Normalized over All Images",
    y=max_subplot_height + 0.015,
    verticalalignment="bottom",
)
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "DTI_channel_sample_global_norm.png");

#### Channel-Wise Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, low-res input, and root squared error
# Normalize by index in the DTI coefficients.
# Reshape and concatenate the dtis in order to compute the quantiles of images with
# different shapes (e.g., the low-res input patch).

cmap = "coolwarm"
dtis = [
    test_vol_viz[viz_subj_id].dti.fr[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.lr[(slice(None), *low_res_slice_idx)],
    test_vol_viz[viz_subj_id].dti.spline[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.diqt[(slice(None), *slice_idx)],
    test_vol_viz[viz_subj_id].dti.abs_error[(slice(None), *slice_idx)],
]

nrows = len(dtis)
ncols = len(channel_names)

# Don't take the absolute max and min values, as there exist some extreme (e.g., > 3
# orders of magnitude) outliers. Instead, take some percente quantile.
max_dtis = np.quantile(
    np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.95, axis=1
)
min_dtis = np.quantile(
    np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.05, axis=1
)

max_dtis = np.max(np.abs([max_dtis, min_dtis]), axis=0)
min_dtis = -1 * max_dtis

nrows = len(dtis)
ncols = len(channel_names)

fig = plt.figure(figsize=(12, 7), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)

axs = list()
max_subplot_height = 0
for i_row in range(nrows):
    dti = dtis[i_row]
    axs_cols = list()

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(
            np.rot90(dti[j_col]),
            cmap=cmap,
            interpolation=None,
            vmin=min_dtis[j_col],
            vmax=max_dtis[j_col],
        )
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(dti_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(channel_names[j_col])

        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])

        axs_cols.append(ax)

    axs.append(axs_cols)

# Place colorbars on each column.
for j_col in range(ncols):

    full_col_ax = [axs[i][j_col] for i in range(nrows)]

    color_norm = mpl.colors.Normalize(vmin=min_dtis[j_col], vmax=max_dtis[j_col])

    color_mappable = mpl.cm.ScalarMappable(norm=color_norm, cmap=cmap)
    cbar = fig.colorbar(
        color_mappable,
        ax=full_col_ax,
        location="top",
        orientation="horizontal",
        pad=0.01,
        shrink=0.85,
    )
    cbar.ax.tick_params(labelsize=8, rotation=35)
    cbar.ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:g}"))
#     cbar.ax.ticklabel_format(scilimits=(3, -3), useOffset=False)

plt.suptitle(
    "DTI Channel Breakdown, Channel-Wise Normalization",
    y=max_subplot_height + 0.01,
    verticalalignment="bottom",
)
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "DTI_channel_sample_channel_wise_norm.png");

### FA Map

In [ ]:
# Slice locations for 2D visualization

half_fr_space_shape = np.floor(
    np.asarray(test_vol_viz[viz_subj_id].fa.fr.shape) // 2
).astype(int)
half_fr_space_shape = half_fr_space_shape.tolist()

slice_indices = [
    (half_fr_space_shape[0], slice(None, None, None), slice(None, None, None)),
    (slice(None, None, None), half_fr_space_shape[1], slice(None, None, None)),
    (slice(None, None, None), slice(None, None, None), half_fr_space_shape[2]),
]

low_res_slice_indices = list()
for slice_idx_i in slice_indices:
    slice_coords = tuple()
    for s in slice_idx_i:
        slice_coords = slice_coords + (
            int(np.floor(s / downsample_factor)) if isinstance(s, int) else s,
        )
    low_res_slice_indices.append(slice_coords)

print(slice_indices)
print()
print(low_res_slice_indices)

row_names = [
    "Saggital",
    "Coronal",
    "Horizontal",
]

model_names = [
    "Full-Res",
    "Low-Res Input",
    f"Spline Order {exp_params.spline.order}",
    "Predicted",
    #     "Absolute Error\nFR vs. Predicted",
]

nrows = len(row_names)
ncols = len(model_names)

In [ ]:
imgs = dict()
for i_row, slice_i in enumerate(slice_indices):
    low_res_slice_i = low_res_slice_indices[i_row]
    imgs[i_row] = dict()
    col_imgs = [
        test_vol_viz[viz_subj_id].fa.fr[(*slice_i,)],
        test_vol_viz[viz_subj_id].fa.lr[(*low_res_slice_i,)],
        test_vol_viz[viz_subj_id].fa.spline[(*slice_i,)],
        test_vol_viz[viz_subj_id].fa.diqt[(*slice_i,)],
        #         test_vol_viz[viz_subj_id].fa.abs_error[(*slice_i,)],
    ]

    imgs[i_row].update(tuple(enumerate(col_imgs)))

#### Global Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, and root squared error

cmap = "gist_gray"

# Don't take the absolute max and min values, as there exist some extreme (e.g., > 3
# orders of magnitude) outliers. Instead, take some percente quantile.
# Concatenate the images in order to compute the quantiles of images with
# different shapes (e.g., the low-res input patch).
max_intensity = np.quantile(
    np.concatenate(
        [imgs[i][j] for (i, j) in itertools.product(range(nrows), range(ncols))],
        axis=None,
    ),
    0.95,
)
min_intensity = np.quantile(
    np.concatenate(
        [imgs[i][j] for (i, j) in itertools.product(range(nrows), range(ncols))],
        axis=None,
    ),
    0.05,
)

fig = plt.figure(figsize=(5 * 1.5, 3 * 1.5), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)
axs = list()
max_subplot_height = 0
for i_row in range(nrows):
    img_row = imgs[i_row]

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(
            np.rot90(img_row[j_col]),
            cmap=cmap,
            interpolation=None,
            vmin=min_intensity,
            vmax=max_intensity,
        )
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(row_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(model_names[j_col], size="small")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        axs.append(ax)

color_norm = mpl.colors.Normalize(vmin=min_intensity, vmax=max_intensity)
fig.colorbar(
    mpl.cm.ScalarMappable(norm=color_norm, cmap=cmap),
    ax=axs,
    location="right",
    fraction=0.1,
    pad=0.03,
)
plt.suptitle(
    "FA Maps, Normalized over All Images",
    y=max_subplot_height + 0.015,
    verticalalignment="bottom",
)
if not disable_fig_save:
    plt.savefig(experiment_results_dir / "fa_sample_global_norm.png");

---

## End Experiment

In [ ]:
pl_logger.experiment.flush()
# Close tensorboard logger.
# Don't finalize if the experiment was for debugging.
if "debug" not in EXPERIMENT_NAME.casefold():
    pl_logger.finalize("success")
    # Experiment is complete, move the results directory to its final location.
    if experiment_results_dir != final_experiment_results_dir:
        print("Moving out of tmp location")
        experiment_results_dir = experiment_results_dir.rename(
            final_experiment_results_dir
        )
        log_txt_file = experiment_results_dir / log_txt_file.name